In [5]:
print("test")

test


In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import keras

# 1. Stała wartość seed
seed = 42

# 2. Ustawienie seedów dla reprodukowalności
os.environ["PYTHONHASHseed"] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

# 3. Wymuszenie deterministycznych operacji w TensorFlow
os.environ["TF_DETERMINISTIC_OPS"] = "1"
tf.config.experimental.enable_op_determinism()

# 4. Parametryzacja
train_dir = "plants_train"
val_dir = "plants_test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,        # Skaluje wartości pikseli z zakresu 0-255 na zakres 0-1 (taki zakres jest łatwiejszy do obróbki dla sieci neuronowych)
    rotation_range = 20,        # Losowo obraca obrazy o kąt od -20 do 20 stopni (pomaga modelowi nauczyć się rozpoznawać obiekty pod różnymi kątami)
    zoom_range = 0.2,           # Losowo przybliża obrazy o maksymalnie 20% (symulacja różnych odległości od obiektu)
    horizontal_flip = True,     # Losowo odbija obrazy poziomo (pomaga modelowi uczyć się, że kierunek nie ma znaczenia)
    validation_split = 0.2      # Dzieli dane na dane treningowe i walidacyjne (20% danych na walidację, 80% na trening)
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

from tensorflow.keras.applications import MobileNetV2

# 1. base_model = MobileNetV2(...):
# MobileNetV2 to funkcja, która tworzy model MobileNetV2 w Keras. W tym przypadku tworzysz model, który będzie pełnił rolę "bazy", na której będziesz budował resztę swojego modelu.
base_model = MobileNetV2(

    # 2. weights = "imagenet":
    # Używasz pretrenowanych wag z ImageNet. ImageNet to ogromny zbiór danych zawierający miliony obrazów z różnymi klasami obiektów (np. psy, koty, samochody itd.). Model MobileNetV2 jest już wytrenowany na tym zbiorze, co oznacza, że zna już podstawowe cechy wizualne, takie jak kształty, kolory, tekstury, itp.
    # Dzięki temu, zamiast trenować model od podstaw (co jest bardzo czasochłonne), możesz wykorzystać tę wcześniej zdobytą wiedzę i dostosować model do swojego zadania.
    weights = "imagenet",

# -------
    # 3. include_top = False:
    # include_top mówi, czy chcesz, aby na końcu modelu znajdowały się warstwy odpowiedzialne za klasyfikację (tzw. "top layers").
    # False oznacza, że usuwasz te warstwy. W rezultacie model MobileNetV2 będzie miał tylko część "feature extraction" (czyli warstwy odpowiedzialne za wydobywanie cech z obrazów), ale nie będzie końcowej warstwy, która wykonuje klasyfikację.
    # Robisz to, ponieważ chcesz dostosować model do własnych danych. Zamiast korzystać z domyślnej klasyfikacji dla ImageNet, chcesz dodać własne warstwy klasyfikacyjne (np. z inną liczbą klas, odpowiadającą Twojemu zbiorowi danych).
    include_top = False,

# -------
    # 4. input_shape = (*IMG_SIZE, 3):
    # input_shape ustawia kształt danych wejściowych, które model będzie przyjmować.
    # *IMG_SIZE to rozmiar obrazów, który określiłeś wcześniej (224x224).
    # Oznacza to, że obrazy wejściowe będą miały wymiary 224x224.
    # 3 na końcu oznacza liczbę kanałów kolorów w obrazie (3 = RGB) - oznacza to że są to obrazki kolorowe. Więc obraz wejściowy będzie miał wymiary 224x224x3 (wysokość, szerokość, liczba kanałów kolorów).
    input_shape = (*IMG_SIZE, 3)
)

# -------
# 5. base_model.trainable = False:
# Po załadowaniu modelu, ustawiasz trainable = False dla warstw bazy (MobileNetV2).
# Oznacza to, że nie będziesz trenować wag wstępnie wytrenowanego modelu.
# Jest to częsta praktyka w transfer learning. Kiedy korzystasz z pretrenowanego modelu, na ogół nie trenujesz go od nowa, ponieważ ma on już wyuczoną wiedzę o podstawowych cechach wizualnych.
# Dzięki temu model będzie działał szybciej, ponieważ tylko Twoje dodatkowe warstwy (które dodasz później) będą trenowane, a nie cały model.
# Umożliwia to również oszczędność pamięci i zmniejsza ryzyko przeuczenia.
base_model.trainable = False

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Bierzemy wyjście z warstwy bazowego modelu (MobileNetV2), które zawiera cechy obrazu
x = base_model.output

# Warstwa GlobalAveragePooling2D wykonuje globalne średnie pooling dla każdego kanału (filtra) obrazu
# Zmienia wymiary z (wysokość, szerokość, liczba filtrów) na (liczba filtrów)
x = GlobalAveragePooling2D()(x)

# Warstwa Dense wprowadza dodatkową abstrakcję, gdzie mamy 128 neuronów.
# Funkcja aktywacji Leaky ReLU pozwala na mały przepływ wartości ujemnych, unikając martwych neuronów.
x = Dense(128, activation = "leaky_relu")(x)

# Warstwa Dense, która odpowiada za klasyfikację.
# Liczba neuronów to liczba klas w danych (np. 12 klas roślin).
# Funkcja aktywacji Softmax daje prawdopodobieństwo dla każdej klasy.
outputs = Dense(
    train_generator.num_classes,  # Liczba klas
    activation = "softmax"        # Daje prawdopodobienstwa
)(x)
"""
test
"""
# Tworzymy pełny model, który łączy wejście z bazowego modelu (MobileNetV2) oraz wyjście z warstwy klasyfikacyjnej
model = Model(
    inputs = base_model.input,  # Wejście modelu to wejście z bazowego modelu (MobileNetV2)
    outputs = outputs           # Wyjście modelu to wynik z ostatniej warstwy klasyfikacji
)

model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 5
)

Found 864 images belonging to 12 classes.
Found 216 images belonging to 12 classes.


C:\Users\Grzegorz\PycharmProjects\PythonProject\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2539 - loss: 2.3549

C:\Users\Grzegorz\PycharmProjects\PythonProject\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.3449 - loss: 2.0077 - val_accuracy: 0.5185 - val_loss: 1.3493
Epoch 2/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.5810 - loss: 1.2212 - val_accuracy: 0.6389 - val_loss: 1.1188
Epoch 3/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.6597 - loss: 1.0001 - val_accuracy: 0.6759 - val_loss: 1.0105
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.6991 - loss: 0.8402 - val_accuracy: 0.7037 - val_loss: 0.9111
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.7488 - loss: 0.7120 - val_accuracy: 0.6759 - val_loss: 0.9369
